# Using KDB+
* * *
## Load Database
### From Original CSVs

This action only has to be performed once. If you already have the splayed tables saved somewhere, go to section "From Splayed Tables" to load them.

In [1]:
/ Load price tick file
tick:("SIDTFIFIFIS"; enlist"|")0:`:/Users/alfredo.leon/Desktop/findata/data/scale_1000/tick_price_file_no_spaces.csv
/ Create enumeration for table (this is required to create a splayed table and then a partitioned table)
tickenum: .Q.en[`:../data/test/raw/tickprice/] tick
/ Sample query
select[10] from tickenum where TimeStamp within 09:00:00.00 09:00:10.00
/ Save table
/rsave `tickenum

Id           SeqNo TradeDate  TimeStamp    TradePrice TradeSize AskPrice AskS..
-----------------------------------------------------------------------------..
Security_359 1     2022.11.21 09:00:00.000 43         6300                   ..
Security_667 1     2022.11.21 09:00:00.000 57         9100                   ..
Security_588 1     2022.11.21 09:00:00.000 27         6800                   ..
Security_371 1     2022.11.21 09:00:00.000                      23.0938  9600..
Security_661 1     2022.11.21 09:00:01.000                                   ..
Security_591 1     2022.11.21 09:00:01.000                                   ..
Security_794 1     2022.11.21 09:00:01.000 26         700                    ..
Security_602 1     2022.11.21 09:00:01.000 58         8100                   ..
Security_539 1     2022.11.21 09:00:02.000                                   ..
Security_27  1     2022.11.21 09:00:02.000 31         5600                   ..


In [1]:
select last SeqNo from tickenum

[0;31mparse error[0m: [0;31m%[0m

In [4]:
/ Load base tick file
base:("SSSSS"; enlist"|")0:`:/Users/alfredo.leon/Desktop/findata/data/scale_1000/tick_base_file.csv
/ Create enumeration for table (this is required to create a splayed table and then a partitioned table)
baseenum: .Q.en[`:../data/test/raw/basetick/] base
/ Sample query
select[5] from baseenum
/ Save table
/rsave `baseenum

Id         Ex Descr                          SIC             Cu 
----------------------------------------------------------------
Security_0 LN 'Financial security number: 0' COMPUTERS       FFR
Security_1 NY 'Financial security number: 1' PHARMACEUTICALS FFR
Security_2 O  'Financial security number: 2' SOFTWARE        USD
Security_3 O  'Financial security number: 3' SOFTWARE        USD
Security_4 AM 'Financial security number: 4' COMPUTERS       GBP


### From Splayed Tables

In [ ]:
/ Load the tables from saved splayed table using get to save into variable.
/baseenum2:get `baseenum / This should be the path to the foldeer with data.
/priceenum2:get `tickenum

In [ ]:
/ Pending: Figure out how to partition tables and save/load those

## Benchmark Queries

1. Get all ticks for a specified set of 100 securities for a specified three hour time period on a specified trade date.

2. Determine the volume weighted price of a security considering only the ticks in a specified three hour interval

3. Determine the top 10 percentage losers for the specified date on the specified exchanges sorted by percentage loss. The loss is calculated as a percentage of the last trade price of the previous day.

4. Determine the top 10 most active stocks for a specified date sorted by cumulative trade volume by considering all trades

5. Find the most active stocks in the "COMPUTER" industry (use SIC code)

6. Find the 10 stocks with the highest percentage spreads. Spread is the difference between the last ask-price and the last bid-price. Percentage spread is calculated as a percentage of the mid-point price (average of ask and bid price).

1. Get all ticks for a specified set of 100 securities for a specified three hour time period on a specified trade date.

In [2]:
distinct select TradeDate from tickenum

TradeDate 
----------
2022.11.21
2022.11.22
2022.11.23
2022.11.24
2022.11.25


In [2]:
secs : select[101] Id from tickenum
securities: (exec Id from secs)  / This is a list
select from tickenum where Id in securities, TradeDate=2022.11.21, TimeStamp within 09:00:00.00 12:00:00.00

Id           SeqNo TradeDate  TimeStamp    TradePrice TradeSize AskPrice AskS..
-----------------------------------------------------------------------------..
Security_359 1     2022.11.21 09:00:00.000 43         6300                   ..
Security_667 1     2022.11.21 09:00:00.000 57         9100                   ..
Security_588 1     2022.11.21 09:00:00.000 27         6800                   ..
Security_371 1     2022.11.21 09:00:00.000                      23.0938  9600..
Security_661 1     2022.11.21 09:00:01.000                                   ..
Security_591 1     2022.11.21 09:00:01.000                                   ..
Security_794 1     2022.11.21 09:00:01.000 26         700                    ..
Security_602 1     2022.11.21 09:00:01.000 58         8100                   ..
Security_539 1     2022.11.21 09:00:02.000                                   ..
Security_27  1     2022.11.21 09:00:02.000 31         5600                   ..
Security_634 1     2022.11.21 09:00:02.0

2. Determine the volume weighted price of a security considering only the ticks in a specified three hour interval.

In [2]:
select sum(TradePrice * TradeSize) % sum(TradeSize) from tickenum where Id=`Security_1, TradeDate=2022.11.21, TimeStamp within 09:00:00.00 12:00:00.00

TradePrice
----------
3.99304   


3. Determine the top 10 percentage losers for the specified date on the specified exchanges sorted by percentage loss. The loss is calculated as a percentage of the last trade price of the previous day.

In [3]:
select[10] from
    `LossPercentage xasc `Id`LossPercentage xcol
    select Id, (ClosingPriceToday-ClosingPricePreviousDay)*100%ClosingPricePreviousDay from(
        `Id`ClosingPriceToday`ClosingPricePreviousDay xcol
        (
            (`Id`ClosingPriceToday xcol select first TradePrice by Id from (`TimeStamp xdesc 
                select Id, TradePrice, TimeStamp 
                from tickenum
                where TradeDate=2022.11.22, TradePrice>0.0)) ij
            (`Id`ClosingPricePreviousDay xcol select first TradePrice by Id from (`TimeStamp xdesc 
                select Id, TradePrice, TimeStamp 
                from tickenum 
                where TradeDate=2022.11.23, TradePrice>0.0))
        )
    )

Id           LossPercentage
---------------------------
Security_382 -70           
Security_96  -62.5         
Security_999 -59.09091     
Security_322 -33.33333     
Security_524 -26.66667     
Security_656 -25.80645     
Security_95  -23.52941     
Security_19  -21.05263     
Security_980 -19.79167     
Security_799 -15.58442     
